#  LangChain의 개념과 주요 컴포넌트 이해

---

## LangChain이란 

- **LangChain**은 LLM 기반 애플리케이션 개발을 위한 프레임워크

- **Chain**은 작업을 순차적으로 실행하는 파이프라인 구조를 제공

- **Agent**는 자율적 의사결정이 가능한 실행 단위

## LangChain 컴포넌트 

- **언어 처리 기능**은 LLM/ChatModel이 중심이 되며, Prompt와 Memory로 대화를 관리

- **문서 처리와 검색**은 Document Loader, Text Splitter, Embedding, Vectorstore가 담당

- **모듈성**이 핵심 특징으로, 독립적인 컴포넌트들을 조합해 RAG와 같은 복잡한 시스템을 구현 가능 

---

# 환경 설정 및 준비

In [ ]:
# 라이브러리 설치
# uv add ipykernel python-dotenv langchain langchain-openai

In [ ]:
# .env 파일 설정
# OPENAI_API_KEY=your_openai_api_key

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

# 1. 모델 (Models)
- LLM, ChatModel 등으로 구분
- OpenAI, Anthropic, Google 등 다양한 모델을 지원
- 텍스트 생성, 대화, 요약 등의 작업을 수행

In [ ]:
from langchain_openai import ChatOpenAI

# OpenAI 모델을 사용하여 대화 생성
model = ChatOpenAI(model="gpt-4.1-nano")

# 모델에 메시지를 보내고 응답을 받기
response = model.invoke("안녕하세요!")

In [ ]:
# 응답 객체(AIMessage): 메시지(content)와 메타데이터(response_metadata 등)를 포함
response

In [ ]:
# 응답 객체의 메시지 내용 출력
print("답변: ", response.content)

In [ ]:
# 응답 객체의 메타데이터 출력
print("메타데이터: ", response.response_metadata)

# 2. 메시지 (Messages)
- Chat Model에서 사용할 수 있는 통합된 메시지 형식을 제공
- 각 모델 제공자의 특정 메시지 형식을 신경 쓰지 않고도 다양한 채팅 모델을 활용 가능

`1. HumanMessage`
- 사용자 역할에 해당 (user, human 등)
- 사용자의 입력을 처리

In [ ]:
from langchain_core.messages import HumanMessage

# 사용자 메시지 생성
human_message = HumanMessage(content="Glory를 한국어로 번역해주세요.")

# 번역 요청 및 응답 받기
response = model.invoke([human_message])  # 메시지 리스트로 전달

# 답변 출력
print("답변: ", response.content)

In [ ]:
# 문자열을 입력하면, 자동으로 HumanMessage로 변환하여 요청
model.invoke("Glory를 한국어로 번역해주세요.")

`2. AIMessage`
- AI 모델의 응답을 표현


In [ ]:
# AI 모델의 응답 객체를 출력 
response

In [ ]:
# 응답 객체의 자료형 확인
type(response)

In [ ]:
# 모델 응답 텍스트 부분을 출력
response.content

In [ ]:
# 토큰 사용량 출력
response.usage_metadata

`3. SystemMessage`
- 시스템 역할에 해당 (system, developer 등)
- AI 모델의 동작과 제약사항을 정의하는데 사용


In [ ]:
from langchain_core.messages import SystemMessage 

# 시스템 메시지 생성
system_msg = SystemMessage(content="당신은 영어를 한국어로 번역하는 AI 어시스턴트입니다.")

# 메시지 객체 확인
system_msg

In [ ]:
# 번역 요청 (HumanMessage)과 시스템 메시지(SystemMessage)를 함께 사용
human_message = HumanMessage(content="Glory")
messages = [system_msg, human_message]

# 모델에 메시지를 보내고 응답 받기
response = model.invoke(messages)

# 답변 출력
print("답변: ", response.content)

# 3. 프롬프트 템플릿 (Prompt Template)
- 프롬프트 템플릿을 통해 일관된 입력 형식을 제공
    1. 사용자의 입력과 파라미터를 언어 모델이 이해할 수 있는 형태로 변환하는 도구
    2. 언어 모델에게 전달할 지시문을 만드는 틀
- 변수를 포함한 동적 프롬프트 생성이 가능
    1. 모든 템플릿은 딕셔너리 형태의 입력을 받아서 처리
    2. 출력은 PromptValue 형태로 반환되며, 이는 문자열이나 메시지 리스트로 변환 가능

`1. 문자열 프롬프트 템플릿 (String PromptTemplate)`
- 가장 기본적인 형태
- 단일 문자열을 형식화하는데 사용

In [ ]:
from langchain_core.prompts import PromptTemplate

# 템플릿 생성 
# "{topic}에 대한 이야기를 해줘"라는 템플릿을 사용하여
# topic이라는 변수를 포함하는 프롬프트를 생성
template = PromptTemplate.from_template("{topic}에 대한 이야기를 해줘")

# 템플릿 사용
# "고양이"라는 주제를 사용하여 프롬프트 생성
# invoke 메서드를 통해 템플릿에 값을 전달
prompt = template.invoke({"topic": "고양이"})

# 템플릿 출력
prompt

`2. 채팅 프롬프트 템플릿 (ChatPromptTemplate)`
- 여러 메시지를 포함하는 대화형 템플릿을 만들 때 사용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 채팅 템플릿 생성
# 여기서는 시스템 메시지와 사용자 메시지를 포함하여 정의
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 비서입니다"),
    ("user", "{subject}에 대해 설명해주세요")
])

# 템플릿 사용
prompt = template.invoke({"subject": "인공지능"})

# 출력
prompt

`3. 메시지 플레이스홀더 (MessagesPlaceholder)`
- 기존 메시지 목록을 템플릿의 특정 위치에 삽입할 때 사용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 메시지 플레이스홀더가 있는 템플릿
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 비서입니다"),
    MessagesPlaceholder("chat_history")   # 채팅 기록을 플레이스홀더로 사용 (예: 이전 대화 내용) -> 이 위치에 메시지 목록을 추가할 수 있음
])

# 템플릿 사용
prompt = template.invoke({
    "chat_history": [
        HumanMessage(content="안녕하세요! 제 이름은 스티브입니다."),
        AIMessage(content="안녕하세요! 무엇을 도와드릴까요?"),
        HumanMessage(content="제 이름을 기억하나요?")
        ]
})

# 출력
prompt.messages

# 4. 출력 파서 (Output Parser)
1. **역할과 기능**
    - 모델의 텍스트 출력을 구조화된 데이터로 변환
    - 채팅 모델과 LLM의 출력을 정규화
    - 다운스트림 작업을 위한 데이터 형식 변환

2. **사용 시 고려사항**
    - OpenAI function calling과 같은 기능이 있는 경우, 해당 기능을 우선 사용

`(1) StrOutputParser`

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 기본적인 문자열 파서 사용
parser = StrOutputParser()

# 프롬프트 템플릿 설정
prompt = PromptTemplate.from_template("도시 {city}의 특징을 알려주세요")

# 모델 정의
model = ChatOpenAI(model='gpt-4.1-nano')

# 체인 구성
chain = prompt | model | parser

# 체인 실행
result = chain.invoke({"city": "서울"})

# 결과 출력
print(result)

`(2) 구조화된 출력 (with_structured_output 메소드)`

In [ ]:
from pydantic import BaseModel, Field

# Pydantic 클래스로 출력 구조를 정의
class CityInfo(BaseModel):
    name: str = Field(description="도시 이름")
    description: str = Field(description="도시의 특징")

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template("도시 {city}의 특징을 알려주세요.")

# 모델 생성 및 구조화된 출력 바인딩
model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
structured_model = model.with_structured_output(CityInfo)

# 프롬프트와 모델 체인 연결
chain = prompt | structured_model

# 체인 실행
result = chain.invoke({"city": "서울"})

In [ ]:
# 결과 출력 (CityInfo 객체)
print(type(result))
print("-" * 20)
print(result)
print("-" * 20)
print(f"도시 이름: {result.name}")
print(f"특징: {result.description}")

### [실습] 

- 구조화된 출력을 사용하여, 다음 뉴스 기사에서 언론사, 기사 제목, 기사 내용, 작성자, 작성일을 추출해보세요.

In [ ]:
# 실제 뉴스 기사를 복사하여 사용하세요.

news_article = """
"복잡한 납세 등 AI가 돕는다"…정부, 초거대 AI 공공서비스 개발
입력2025.06.20. 오후 12:00  수정2025.06.20. 오후 1:39 

[서울=뉴시스]윤현성 기자 = 정부가 공공분야에 도입할 초거대 인공지능(AI) 기반 국민 편의 서비스 본격 개발에 나선다. 세금 납부 시 생성형 AI 챗봇을 통해 어려운 세무 용어 등에 대한 설명·상담을 듣거나, 대량의 민원업무도 생성형 AI 기반 분석으로 빠르게 답변·응대하는 등의 형태다.

과학기술정보통신부와 디지털플랫폼정부위원회는 2025년도 '초거대 AI 서비스 개발 지원' 사업을 본격 추진하기 위해 수행기업 공모를 실시한다고 20일 밝혔다. 이 사업은 공공분야에 초거대 AI를 도입·확산하고 이를 통해 행정 효율화, 대국민 서비스 혁신, 사회현안 해결이 가능한 서비스 개발을 목표로 추진된다.

올해는 다양한 공공분야에서 초거대 AI 기술을 통해 국민이 체감할 수 있고 실질적인 변화를 가져올 수 있는 과제를 중점적으로 발굴했다. 중앙부처·지자체·공공기관을 대상으로 1~2월에 과제 공모를 추진했으며 총 5개 과제가 선정됐다.

국민권익위원회의 '생성형 AI 기반 국민소통·민원분석 체계 구축'은 국민소통시스템에 생성형 AI 기반 민원분석 체계를 도입해 민원처리 행정 효율화와 답변품질을 향상시킨다. 이를 통해 대량의 민원업무를 신속·효율적으로 대응해 민원업무의 효율성을 증대하고 국민의 신뢰도 향상에도 기여할 것으로 기대된다.

국세청의 '생성형 AI 기반 국세 상담 지원 서비스'는 납세자가 홈택스 이용 시 전자신고 관련한 문의사항을 즉시 해소할 수 있는 실시간 상담 서비스를 제공한다. 홈택스에 상담전용 AI챗봇을 도입해 전화 상담 시 발생하는 장시간 대기 문제를 해결하고, 어려운 세무 용어 등으로 인한 불편사항을 개선할 예정이다.

산업통상자원부의 '해외인증 공공특화 AI 에이전트 서비스'는 모바일 플랫폼, 소셜네트워크서비스 등 사용자 친화적인 모바일 기반 해외인증 특화 AI 에이전트 서비스를 제공한다. 중소기업이 겪는 수출 관련 애로사항인 해외 인증과 관련된 정보와 질의 응답을 AI기반으로 제공하여 손쉽게 확인할 수 있을 것으로 기대된다.

국민건강보험공단의 '에이전틱 AI기반 전국민 맞춤형 민원 상담 서비스'는 국민 생활과 편익에 직결되는 건강보험 민원 상담업무에 AI를 도입해 24시간 개인 맞춤형 민원 상담 서비스를 구현한다. 기존의 전화 상담 방식의 대기 시간 문제 등을 해소하고, 고객센터 집중 상담을 분산시켜 업무의 효율성을 향상시키는데 기여할 것으로 전망이다.

한국지역정보개발원의 '지방재정 지능화 서비스'는 e호조+, 지방재정365 등 지방재정서비스에 생성형 AI를 접목시켜 대국민, 공무원 등 각자의 요구에 부합하는 융복합 재정정보서비스 환경을 제공하고자 한다. 이 서비스가 도입되면 지방정부의 사회현안 해결을 위한 정책 수립의 적시성 향상 및 전문성 확보, 지자체 정보 접근성 강화로 대국민의 사회 참여 기회를 확대할 것으로 예상된다.

이번 사업은 19일 국민권익위원회와 국세청 과제의 민간 전문기업 조달 공고를 시작으로, 5개 과제별 서비스 개발지원 사업이 순차적으로 입찰공고될 예정이다.

김경만 과기정통부 인공지능기반정책관은 "선정된 과제에 대해 민·관 협력을 기반으로 행정 현장의 변화와 국민이 체감할 수 있는 성과가 창출·확산될 수 있도록 적극 지원할 계획"이라며 "개발된 서비스는 공공분야에서 행정업무의 효율성을 높이고 대국민 서비스 품질을 높이는 데 실질적으로 기여할 수 있도록 민간에서도 많은 관심을 가져주실 것을 부탁드린다"고 말했다.

이승현 디플정위 인공지능·플랫폼혁신국장은 "이번 사업은 노동, 복지, 민원 등 다양한 공공 분야에 AI를 도입·활용하는데 중요한 역할을 하고 있다"며 "올해도 AI를 활용해 사회문제를 해결하고, 대국민 서비스를 혁신적으로 개선할 수 있는 서비스가 개발되길 기대한다"고 전했다．

윤현성 기자(hsyhs@newsis.com)
"""

In [ ]:
# 여기에 코드를 작성하세요. 

<details>
<summary>💡 정답 보기</summary>

```python
# 출력 스키마 정의
from pydantic import BaseModel, Field

class NewsSummary(BaseModel):
    title: str = Field(description="뉴스 제목")
    content: str = Field(description="뉴스 본문")
    author: str = Field(description="작성자")
    published_date: str = Field(description="작성일")
    agency: str = Field(description="뉴스 제공 기관")

# 프롬프트 템플릿 생성
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 기사를 요약하는 AI 어시스턴트입니다."),
    ("user", "다음 뉴스 기사를 요약해주세요:\n\n{news_article}\n\n"
              "제목, 본문, 작성자, 작성일, 제공 기관을 포함한 요약을 생성해주세요.")
])

# 모델 생성
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# 구조화된 출력 모델 생성
structured_model = model.with_structured_output(NewsSummary)

# 체인 구성
chain = prompt | structured_model

# 체인 실행
result = chain.invoke({"news_article": news_article})

# 결과 출력
print("제목: ", result.title)
print("본문: ", result.content)
print("작성자: ", result.author)
print("작성일: ", result.published_date)
print("제공 기관: ", result.agency)
```
</details>
